<a href="https://colab.research.google.com/github/saberforce/Portfolio-Component-Risk-Return-Landscape/blob/main/coingeckoinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install pycoingecko


In [15]:
import pandas as pd
import requests
import plotly.express as px

In [16]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()


In [17]:
coindict={}
portfoliolist=['bitcoin', 'ethereum', 'wmatic','chainlink', 'uniswap','the-sandbox',
           'decentraland','the-graph','aave', 'wrapped-usdc']

# Allocated % Inside The Portfolio In Ordered Sequence
portfoliolistportion=[0.2,0.2,0.1,0.1,0.05,0.05,0.05,0.05,0.05,0.15]

# For Each Coin Get A List Of Prices Over The Last 365 Days
def get_asset_price(assetlist, assetdict):
  for coin in assetlist:
    asset_price=cg.get_coin_market_chart_by_id(id=coin,vs_currency='usd',days='364')
    pricelist=[]
    num_of_prices=len(asset_price['prices'])
    for k in range(num_of_prices):

      coinprice=(asset_price['prices'][k][1])
      pricelist.append(coinprice)


 #Prep data - deficient data count for some coins
    assetdict[f'{coin}']=pd.Series(pricelist)

  return assetdict

# Getting A List of Returns Derived From Daily Prices For Each Coin, 
# The Expected Mean Of Return And The Standard Deviation For Each Coin
# Deriving The Portfolio Mean And Portfolio Standard Deviation

def calculate_stats(assetdict, assetlist, assetlistportion, df1, list_of_mean, list_of_std):
  n=0
  portfolio_mean=0
  portfolio_std=0

  returnlist = [x+"_returns" for x in assetlist]

  for k,v in assetdict.items():
    df1[f'{k}']=v
    df1[returnlist[n]]=df1[f'{k}'].pct_change(1)
    
    asset_mean=df1[returnlist[n]].mean()*100
    asset_std=df1[returnlist[n]].std()*100

    portfolio_mean=portfolio_mean+(assetlistportion[n]*asset_mean)

    portfolio_std=portfolio_std+(assetlistportion[n]*asset_std)

    list_of_mean.append(asset_mean)
    list_of_std.append(asset_std)
    n=n+1
  return portfolio_mean, portfolio_std

def plot_scatter(dfa,x,y):
  fig = px.scatter(dfa, x, y,
                 hover_name="name", 
                 color= "name", 
                 size= abs(dfa['mean%']),
                 size_max=50,
                 text="name",
                 title="Risk Return Landscape",



                 )


  fig.show()

def main(coindict, portfoliolist, portfoliolistportion):

  a=get_asset_price(portfoliolist, coindict)
  df_asset_price=pd.DataFrame()
  meanlist=[] #list is mutable by method
  stdlist=[]

  portfolio_mean, portfolio_std= calculate_stats(coindict, portfoliolist, 
                                                 portfoliolistportion, 
                                                 df_asset_price, meanlist,stdlist)
  
  portfoliolist.append('portfolio')
  meanlist.append(portfolio_mean)
  stdlist.append(portfolio_std)
  
  # Finalizing The Finished Dataframe Tabulation For 9 Coins And 1 Portfolio Item

  listOfFinals = list(zip(portfoliolist, meanlist, stdlist))

  df_final = pd.DataFrame(listOfFinals, columns = ['name', 'mean%','std%'])
  print(df_final)

  plot_scatter(df_final, x="std%", y="mean%")


if __name__ == "__main__":
  main(coindict, portfoliolist, portfoliolistportion)

            name     mean%       std%
0        bitcoin  0.035634   3.849802
1       ethereum  0.265129   5.125415
2         wmatic  1.042996   9.060163
3      chainlink  0.031148   6.762083
4        uniswap  0.060963   7.129907
5    the-sandbox  1.291977  10.674164
6   decentraland  1.142192  11.786499
7      the-graph -0.193731   7.174745
8           aave -0.064082   6.723616
9   wrapped-usdc  0.023334   2.162860
10     portfolio  0.282933   5.876143
